In [15]:
import pandas as pd
import geopandas as gpd
import openrouteservice

client = openrouteservice.Client(key='5b3ce3597851110001cf624899819d8fdf27474db7f91769a7b64b02')

regions_df = pd.read_csv('../../data/raw/location/sa2_to_rental_suburb_groups.csv')
regions_df['geometry'] = gpd.GeoSeries.from_wkt(regions_df['geometry'])

regions_df = gpd.GeoDataFrame(
    regions_df,
    geometry='geometry'
)

MELBOURNE_CENTRAL_COORD = (-37.810272, 144.962646)

In [16]:
from geopy.distance import geodesic

regions_df['centroid'] = regions_df.geometry.centroid

crow_distances = []

# Iterate over the rows in the GeoDataFrame
for index, row in regions_df.iterrows():
    # Extract the latitude and longitude of the centroid
    centroid_coords = (row['centroid'].y, row['centroid'].x)  # y is latitude, x is longitude
    
    # Calculate the straight-line distance (Haversine distance)
    distance = geodesic(centroid_coords, MELBOURNE_CENTRAL_COORD).kilometers  # Distance in kilometers
    crow_distances.append(distance)

# Add the calculated distances to the GeoDataFrame
regions_df['crow_distance_to_cbd'] = crow_distances

regions_df

,Unnamed: 0,geometry,suburbs,regions,code,centroid,crow_distance_to_cbd
0,0,"POLYGON ((144.97018 -37.8607, 144.96997 -37.86...",Albert Park-Middle Park-West St Kilda,"['Albert Park', 'St Kilda - West']","[206051128, 206051514]",POINT (144.96816 -37.8509),4.535857
1,1,"POLYGON ((144.79015 -37.86216, 144.78952 -37.8...",Altona,"['Altona', 'Altona North']","[213021341, 213021343]",POINT (144.82509 -37.84855),12.833585
2,2,"POLYGON ((145.01167 -37.85357, 145.01177 -37.8...",Armadale,['Armadale'],[206061135],POINT (145.02071 -37.85675),7.261986
3,3,"POLYGON ((145.12549 -38.07134, 145.12536 -38.0...",Aspendale-Chelsea-Carrum,"['Carrum - Patterson Lakes', 'Edithvale - Aspe...","[208031189, 208031186, 208031185]",POINT (145.12447 -38.05298),30.465132
4,4,"POLYGON ((147.57603 -37.83022, 147.57594 -37.8...",Bairnsdale,['Bairnsdale'],[205021081],POINT (147.61399 -37.82915),233.444538
...,...,...,...,...,...,...,...
139,139,"POLYGON ((144.86314 -37.79333, 144.8632 -37.79...",West Footscray,['West Footscray - Tottenham'],[213031351],POINT (144.86451 -37.80201),8.690401
140,140,"POLYGON ((144.99578 -37.47133, 144.99981 -37.4...",Whittlesea,['Whittlesea'],[209041225],POINT (145.10196 -37.49791),36.783726
141,141,"POLYGON ((144.887 -37.85079, 144.8873 -37.8501...",Williamstown,['Williamstown'],[213021346],POINT (144.8848 -37.85911),8.738097
142,142,"POLYGON ((146.77394 -36.12801, 146.77189 -36.1...",Wodonga,"['Wodonga', 'West Wodonga']","[204031492, 204031073]",POINT (146.80557 -36.11627),249.542456


In [43]:
gdf = regions_df

# distances = []
# durations = []
# geometries = []

for index, row in gdf.iterrows():
    if index < 143:
        continue
    print(index)
    centroid_coords = list(row['centroid'].coords[0])  # Get centroid coordinates
    mc_reversed = tuple(reversed(MELBOURNE_CENTRAL_COORD))

    # Request route between centroid and specific point
    route = client.directions(
        coordinates=[centroid_coords, mc_reversed],
        profile='driving-car',
        format='geojson',
    )
    
    # Extract distance (in meters) and duration (in seconds)
    distance = route['features'][0]['properties']['segments'][0]['distance']
    duration = route['features'][0]['properties']['segments'][0]['duration']
    route_geometry = route['features'][0]['geometry']['coordinates']

    distances.append(distance)
    durations.append(duration)
    geometries.append(route_geometry)

    print(index, distance, duration)


# Add distances and durations to your GeoDataFrame
# gdf['distance_to_cbd'] = distances  # Distance in meters
# gdf['duration_to_cbd'] = durations  # Duration in seconds
# gdf['route_to_cbd'] = geometries

gdf

143
143 10964.9 1038.1


,Unnamed: 0,geometry,suburbs,regions,code,centroid,crow_distance_to_cbd
0,0,"POLYGON ((144.97018 -37.8607, 144.96997 -37.86...",Albert Park-Middle Park-West St Kilda,"['Albert Park', 'St Kilda - West']","[206051128, 206051514]",POINT (144.96816 -37.8509),4.535857
1,1,"POLYGON ((144.79015 -37.86216, 144.78952 -37.8...",Altona,"['Altona', 'Altona North']","[213021341, 213021343]",POINT (144.82509 -37.84855),12.833585
2,2,"POLYGON ((145.01167 -37.85357, 145.01177 -37.8...",Armadale,['Armadale'],[206061135],POINT (145.02071 -37.85675),7.261986
3,3,"POLYGON ((145.12549 -38.07134, 145.12536 -38.0...",Aspendale-Chelsea-Carrum,"['Carrum - Patterson Lakes', 'Edithvale - Aspe...","[208031189, 208031186, 208031185]",POINT (145.12447 -38.05298),30.465132
4,4,"POLYGON ((147.57603 -37.83022, 147.57594 -37.8...",Bairnsdale,['Bairnsdale'],[205021081],POINT (147.61399 -37.82915),233.444538
...,...,...,...,...,...,...,...
139,139,"POLYGON ((144.86314 -37.79333, 144.8632 -37.79...",West Footscray,['West Footscray - Tottenham'],[213031351],POINT (144.86451 -37.80201),8.690401
140,140,"POLYGON ((144.99578 -37.47133, 144.99981 -37.4...",Whittlesea,['Whittlesea'],[209041225],POINT (145.10196 -37.49791),36.783726
141,141,"POLYGON ((144.887 -37.85079, 144.8873 -37.8501...",Williamstown,['Williamstown'],[213021346],POINT (144.8848 -37.85911),8.738097
142,142,"POLYGON ((146.77394 -36.12801, 146.77189 -36.1...",Wodonga,"['Wodonga', 'West Wodonga']","[204031492, 204031073]",POINT (146.80557 -36.11627),249.542456


In [54]:
# skipped index 71
# skipped index 77
# skipped index 100
# skipped index 122
# skipped index 137
# skipped index 142

distances2 = distances.copy()
durations2 = durations.copy()
geometries2 = geometries.copy()


138

In [55]:
len(distances2)

MISSED_INDICES = [71, 77, 100, 122, 137, 142]

for i, j in enumerate(MISSED_INDICES):
    distances2.insert(j, missing_distances[i])
    durations2.insert(j, missing_durations[i])
    geometries2.insert(j, missing_geometries[i])


In [47]:
gdf['distance_to_cbd'] = distances2  # Distance in meters
gdf['duration_to_cbd'] = durations2  # Duration in seconds
gdf['route_to_cbd'] = geometries2

# gdf.to_csv('distances.csv')

In [50]:
gdf[gdf['distance_to_cbd'].isna()]# .sort_values('suburbs', ascending=False)

,Unnamed: 0,geometry,suburbs,regions,code,centroid,crow_distance_to_cbd,distance_to_cbd,duration_to_cbd,route_to_cbd
71,71,"POLYGON ((144.33933 -38.12686, 144.33937 -38.1...",Herne Hill-Geelong West,['Geelong West - Hamlyn Heights'],[203021040],POINT (144.30196 -38.1265),67.840451,NaN,NaN,None
77,77,"POLYGON ((144.35202 -38.01776, 144.35142 -38.0...",Lara,['Lara'],[203021043],POINT (144.37807 -37.94777),53.645139,NaN,NaN,None
100,100,"POLYGON ((144.35253 -38.2417, 144.35289 -38.23...",Ocean Grove-Barwon Heads,"['Ocean Grove', 'Barwon Heads - Armstrong Creek']","[203031490, 203031489]",POINT (144.46811 -38.24837),65.190343,NaN,NaN,None
122,122,"POLYGON ((144.75997 -37.5986, 144.76398 -37.60...",Sunbury,"['Sunbury', 'Sunbury - West', 'Sunbury - South']","[210041540, 210041240, 210041541]",POINT (144.7444 -37.55393),34.352671,NaN,NaN,None
137,137,"POLYGON ((144.71903 -37.87872, 144.71909 -37.8...",Werribee-Hoppers Crossing,"['Hoppers Crossing - North', 'Werribee - West'...","[213051368, 213051362, 213051468, 213051467, 2...",POINT (144.60613 -37.92862),34.008511,NaN,NaN,None
142,142,"POLYGON ((146.77394 -36.12801, 146.77189 -36.1...",Wodonga,"['Wodonga', 'West Wodonga']","[204031492, 204031073]",POINT (146.80557 -36.11627),249.542456,NaN,NaN,None


In [53]:
missing_distances = []
missing_durations = []
missing_geometries = []

for index, row in gdf.iterrows():
    if index not in MISSED_INDICES:
        continue
    print(index)
    centroid_coords = list(row['centroid'].coords[0])  # Get centroid coordinates
    mc_reversed = tuple(reversed(MELBOURNE_CENTRAL_COORD))

    # Request route between centroid and specific point
    route = client.directions(
        coordinates=[centroid_coords, mc_reversed],
        profile='driving-car',
        format='geojson',
        radiuses=[-1,350]
    )
    
    # Extract distance (in meters) and duration (in seconds)
    distance = route['features'][0]['properties']['segments'][0]['distance']
    duration = route['features'][0]['properties']['segments'][0]['duration']
    route_geometry = route['features'][0]['geometry']['coordinates']

    missing_distances.append(distance)
    missing_durations.append(duration)
    missing_geometries.append(route_geometry)

    print(index, distance, duration)

71
71 79505.0 4076.6
77
77 63528.3 3913.6
100
100 109550.5 5750.3
122
122 42937.7 2864.2
137
137 42733.0 2577.0
142
142 306607.1 12400.8
